# 03. Feature Analysis

Given a specific modeling approach, attempt to determine which features are attended for classification results. Additionally, attempt to obtain Shapley values for the features and present on a figure.


In [ ]:
import os
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

data_df = None
if os.path.isfile("data/data.pkl"):
    data_df = pd.read_pickle("data/data.pkl")
else:
    print("Load Data")
data_df.head()

In [ ]:
tbi_df = data_df[data_df["Study"] == "TBI"]
tbi_evc = np.vstack(tbi_df["EVC"])
tbi_class = np.where(tbi_df["Diagnosis"] == "POS", 1, 0)

In [ ]:
state = 100

clf = svm.SVC(
    kernel="sigmoid", class_weight="balanced", probability=True, random_state=state
)

scaler = StandardScaler()
tbi_evc_scaled = scaler.fit_transform(tbi_evc)

x_train, x_test, y_train, y_test = train_test_split(
    tbi_evc_scaled, tbi_class, random_state=state, stratify=tbi_class
)
feature_variance = pd.DataFrame(x_train).var(axis=0)
print("Feature Variance:\n", feature_variance)
clf.fit(x_train, y_train)
# best_model = clf

In [ ]:
svc_params = {
    "C": np.linspace(0.1, 10, 25),
    "kernel": ["linear", "rbf", "sigmoid"],
    "gamma": np.logspace(1e-10, 1, 10),
}

gridsearch = GridSearchCV(
    estimator=clf,
    param_grid=svc_params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=state),
    refit=True,
)
gridsearch.fit(x_train, y_train)
best_model = gridsearch.best_estimator_

In [ ]:
explainer = shap.KernelExplainer(best_model.predict_proba, x_train)
shap_values = explainer.shap_values(x_test)
shap_pos_class = shap_values[:, :, 1]
shap_values_pos = shap.Explanation(
    values=shap_pos_class,
    base_values=explainer.expected_value[1],
    data=x_test,
    feature_names=[f"Feature {i+1}" for i in range(x_test.shape[1])],
)

In [ ]:
print("Best Model Parameters:", gridsearch.best_params_)

y_predict = best_model.predict(x_test)
print(classification_report(y_test, y_predict))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_predict))

importances = permutation_importance(best_model, x_test, y_test, scoring="accuracy")
print("Feature Importances:\n", importances.importances_mean)

shap.summary_plot(
    shap_pos_class,
    x_test,
    feature_names=[f"Feature {i+1}" for i in range(x_test.shape[1])],
)

In [ ]:
shap.plots.heatmap(shap_values_pos)

In [ ]:
shap.plots.bar(shap_values_pos.abs.max(0))

In [ ]:
# Waterfall plots display individual predictions, so they expect a single row
# of an Explanation object as input
sample_ind = 0
shap.waterfall_plot(
    shap.Explanation(
        values=shap_pos_class[sample_ind],
        base_values=explainer.expected_value[1],
        data=x_test[sample_ind],
        feature_names=[f"Feature {i+1}" for i in range(x_test.shape[1])],
    )
)

### LASSO


In [ ]:
from sklearn.linear_model import Lasso, lasso_path, LassoCV
from sklearn.metrics import mean_squared_error

reg = Lasso(alpha=1)
reg.fit(x_train, y_train)
print("R squared training set", round(reg.score(x_train, y_train) * 100, 2))
print("R squared test set", round(reg.score(x_test, y_test) * 100, 2))

# Training data
pred_train = reg.predict(x_train)
mse_train = mean_squared_error(y_train, pred_train)
print("MSE training set", round(mse_train, 2))

# Test data
pred = reg.predict(x_test)
mse_test = mean_squared_error(y_test, pred)
print("MSE test set", round(mse_test, 2))

In [ ]:
print("Computing regularization path using the lasso...")
eps = 5e-2
alphas_lasso, coefs_lasso, _ = lasso_path(x_train, y_train, eps=eps)

plt.figure(1)
for coef_lasso in coefs_lasso:
    l1 = plt.semilogx(alphas_lasso, coef_lasso)

plt.xlabel("alpha")
plt.ylabel("coefficients")
plt.title("Lasso Paths")
plt.axis("tight")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=state)
lasso_cv = LassoCV(cv=skf, random_state=state, max_iter=10000)
lasso_cv.fit(x_train, y_train)
best_alpha = lasso_cv.alpha_
print(f"Best alpha: {best_alpha}")

plt.axvline(x=best_alpha, color="red", linestyle="dotted", label=f"Alpha: {best_alpha}")

lasso_best = Lasso(alpha=best_alpha)
lasso_best.fit(x_train, y_train)
coefficients = lasso_best.coef_
nonzero_indices = np.where(coefficients != 0)[0]

sorted_data = sorted(
    [
        (idx + 1, coef)
        for idx, coef in zip(nonzero_indices, coefficients[nonzero_indices])
    ],
    key=lambda x: abs(x[1]),
    reverse=True,
)
coefficients_df = pd.DataFrame(sorted_data, columns=["Feature Index", "Coefficient"])

print("R squared training set", round(lasso_best.score(x_train, y_train) * 100, 2))
print("R squared test set", round(lasso_best.score(x_test, y_test) * 100, 2))
print("MSE test:", mean_squared_error(y_test, lasso_best.predict(x_test)))

plt.legend()
plt.show()

In [ ]:
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_, ":")
plt.plot(
    lasso_cv.alphas_,
    lasso_cv.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(lasso_cv.alpha_, linestyle="--", color="k", label="alpha: CV estimate")

plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")

# ymin, ymax = 50000, 250000
# plt.ylim(ymin, ymax);

In [ ]:
coefficients_df

In [ ]:
from sklearn.feature_selection import f_regression

f_stat, p_values = f_regression(tbi_evc_scaled, tbi_class)
feature_data = pd.DataFrame(
    {"Feature Index": range(1, len(p_values) + 1), "P-Value": p_values}
)

significant_features = feature_data[feature_data["P-Value"] < 0.05].sort_values(
    by="P-Value"
)
significant_features

In [ ]:
# from pygam import LogisticGAM, s
# import scipy.sparse

# def to_array(self):
#     return self.toarray()

# scipy.sparse.spmatrix.A = property(to_array)

# np.int = np.int32
# np.float = np.float64
# np.bool = np.bool_
# x_t = pd.DataFrame(x_train)
# print(x_t)
# gam = LogisticGAM(verbose=True).fit(x_t, y_train)
# gam.summary()

In [ ]:
# gam_opt = LogisticGAM().gridsearch(x_train, y_train)
# gam_opt.summary()

In [ ]:
# pd.set_option('display.float_format', '{:.10f}'.format)
# feature_data.to_csv('out.csv')